In [1]:
import numpy as np
from scipy.interpolate import approximate_taylor_polynomial

from scipy import special
import matplotlib.pyplot as plt
from pysrc.optimization import solve_planner_problem
from pysrc.sampling import baseline
from pysrc.services.data_service import load_site_data

/Users/ph2696/Projects/project-amazon/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Model hyperparameters
solver = "gurobi"
pee = 7.6
pa = 41.11
num_sites = 1043
T = 200


(zbar_2017, z_2017, forest_area_2017) = load_site_data(num_sites)

# Set productivity parameters using baseline mean
baseline_fit = baseline.sample(
    num_sites=num_sites,
    iter_sampling=10**3,
    chains=5,
    seed=1,
)

theta = baseline_fit.stan_variable("theta").mean(axis=0)
gamma = baseline_fit.stan_variable("gamma").mean(axis=0)

# Computing carbon absorbed in start period
x_2017 = gamma * forest_area_2017

# Solve planner problem
results = solve_planner_problem(
    x0=x_2017,
    z0=z_2017,
    zbar=zbar_2017,
    gamma=gamma,
    theta=theta,
    time_horizon=T,
    price_cattle=pa,
    price_emissions=pee,
)

17:00:47 - cmdstanpy - INFO - compiling stan file /Users/ph2696/Projects/project-amazon/stan_model/baseline.stan to exe file /Users/ph2696/Projects/project-amazon/stan_model/baseline


Data successfully loaded from /Users/ph2696/Projects/project-amazon/data/calibration/hmc


17:00:52 - cmdstanpy - INFO - compiled model executable: /Users/ph2696/Projects/project-amazon/stan_model/baseline
17:01:15 - cmdstanpy - INFO - CmdStan start processing
chain 1 |          | 00:00 Status









chain 1 |▍         | 00:00 Status





chain 1 |▉         | 00:01 Iteration:   1 / 1000 [  0%]  (Sampling)







chain 1 |█▎        | 00:01 Iteration: 100 / 1000 [ 10%]  (Sampling)



chain 1 |█▊        | 00:02 Iteration: 200 / 1000 [ 20%]  (Sampling)





chain 1 |██▎       | 00:02 Iteration: 300 / 1000 [ 30%]  (Sampling)





chain 1 |██▋       | 00:03 Iteration: 400 / 1000 [ 40%]  (Sampling)


chain 1 |███▏      | 00:04 Iteration: 500 / 1000 [ 50%]  (Sampling)








chain 1 |███▋      | 00:04 Iteration: 600 / 1000 [ 60%]  (Sampling)





chain 1 |████      | 00:05 Iteration: 700 / 1000 [ 70%]  (Sampling)





chain 1 |████▌     | 00:05 Iteration: 800 / 1000 [ 80%]  (Sampling)





chain 1 |█████     | 00:06 Iteration: 900 / 1000 [ 90%]  (Sampling)

chain 1 |██████████| 0


17:01:22 - cmdstanpy - INFO - CmdStan done processing.



Solving the optimization problem...
Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2024-10-30
Read LP format model from file /var/folders/d1/k6sr3htd7fdgk614mmq8pwq00000gr/T/tmpbz8a0rvx.pyomo.lp
Reading time = 1.08 seconds
x1: 417600 rows, 834801 columns, 2083271 nonzeros
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 417600 rows, 834801 columns and 2083271 nonzeros
Model fingerprint: 0x116e6e0d
Model has 400 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [7e-04, 1e+03]
  QObjective range [2e+03, 2e+05]
  Bounds range     [3e-06, 1e+00]
  RHS range        [9e-11, 5e-01]
Presolve removed 208947 rows and 209294 columns
Presolve time: 0.33s
Presolved: 208653 rows, 625507 column

# Solving ODE - Replicating Paper

In [27]:
def area(A, A0=50, alpha=8.5, beta=8.08):
    term1 = np.log2(A0) - np.log2(A)
    result = (1 / 4) * special.erf((alpha - term1) / beta) + (3 / 4)
    return result


def LHS(A):
    return -np.log2(area(A))

In [ ]:
degree = 3
area_taylor = approximate_taylor_polynomial(
    area, 25, degree, scale=0.1, order=degree + 2
)

# Define the range for x
x = np.linspace(0.001, 50, 4000)

# Compute the values of the functions
y1 = area(x)
y2 = area_taylor(x)

# Plot the functions
plt.plot(x, y1, label="a(x)")
plt.plot(x, y2, label="a_taylor(x)")
plt.xlabel("A")
plt.ylabel("a")
plt.title("a v.s Taylor Approximation")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from scipy.integrate import solve_ivp


# Set ODE
def diff_eq(A, S):
    dSdA = (S / A) * LHS(A)
    return dSdA


# Set the initial condition S(1)
S1 = [480]
A_start = 1
A_end = 50

A_space = np.linspace(A_start, A_end, 10000)

# Solve the differential equation
sol = solve_ivp(diff_eq, [A_start, A_end], S1, method="RK45", t_eval=A_space)

# Plot the result
plt.plot(sol.t, sol.y[0], label="S(A)")
plt.xlabel("Area (ha)")
plt.ylabel("Number of Species")
plt.title("Solution of the SAR differential equation")
plt.legend()
plt.show()

# Applying ODE to the Amazon

In [6]:
Z_agg = results.Z.sum(axis=1)
F = (zbar_2017.sum() - Z_agg) * 1e9

In [ ]:
100 * (1 - (F[50] / F[0]) ** 0.125)

In [ ]:
# Set ODE
def LHS(x):
    return -np.log2(area(x, A0=F[0]))


def diff_eq(A, S):
    dSdA = (S / A) * LHS(A)
    return dSdA


# Set condition S(A[0])
S0 = [40000]
A_start = F[0]
A_end = F[50]

A_space = np.linspace(A_start, A_end, 10000000)

# Solve the differential equation
sol = solve_ivp(diff_eq, [A_start, A_end], S0, method="RK45", t_eval=A_space)

# Plot the result
plt.plot(sol.t, sol.y[0], label="S(A)")
plt.xlabel("Area (ha)")
plt.ylabel("Number of Species")
plt.title("Solution of the SAR differential equation")
plt.legend()
plt.show()

In [ ]:
(sol.y[0][0] / sol.y[0][-1])

In [ ]:
round(100 * (sol.y[0][-1] - sol.y[0][0]) / sol.y[0][0], 2)

# Alternative application

In [ ]:
# Set ODE
def LHS(x):
    return -np.log2(area(x, A0=50))


def diff_eq(A, S):
    dSdA = (S / A) * LHS(A)
    return dSdA


# Set condition S(A[0])
S1 = [100]
A_start = 1
A_end = F[0]

A_space = np.linspace(A_start, A_end, 1000000)

# Solve the differential equation
sol = solve_ivp(diff_eq, [A_start, A_end], S1, method="RK45", t_eval=A_space)

# Plot the result
plt.plot(sol.t, sol.y[0], label="S(A)")
plt.xlabel("Area (ha)")
plt.ylabel("Number of Species")
plt.title("Solution of the SAR differential equation")
plt.legend()
plt.show()

In [ ]:
sol.y[0][-1]